This program will parse Sabanci's website for the courses and it will notify you via SMS when there is an empty seat

First we install and import *request* to access urls

In [1]:
!apt-get install requests

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package requests


In [0]:
import requests

Then we install and import *lxml* which allows us to easily parse html

In [4]:
!pip install lxml

    100% |████████████████████████████████| 5.8MB 5.0MB/s 


In [0]:
import lxml
from lxml import html

Now we get the course code number of the courses we want to check

In [0]:
MATH = 11033
MR = list(range(11035, 11035 + 12))
MR

In [7]:
CS = 11063
CR = [11107, 11108, 11149, 11150, 11155, 11156]
CR


[11107, 11108, 11149, 11150, 11155, 11156]

In [8]:
CLASSES = [MATH,CS]
CLASSES

[11033, 11063]

We install and import *twilio* In order to send SMS to my phone number

In [9]:
!pip install twilio

    100% |████████████████████████████████| 901kB 9.5MB/s 
    100% |████████████████████████████████| 286kB 8.8MB/s 
  Running setup.py bdist_wheel for pysocks ... - done
  Stored in directory: /root/.cache/pip/wheels/22/5c/b5/12e0dfdfa85bea67b23628b6425fae715c687e947a45ee3df9
Successfully built pysocks


We define a function called sendSMS which will send an SMS from twilio's registered phone number to my phone number

In [0]:
from twilio.rest import Client
def sendSMS(msg):
  account_sid = "AC1d933a532f82a7a93a2a888bf51d4c49"
  auth_token = "ce3713fe7fcf2ac91702719738370e72"
  client = Client(account_sid,auth_token)

  client.messages.create(
      to="+90 551 554 62 74",
      from_="+15012388272",
      body=msg)

Now we can define a fuction checkSeats which will check the given list of course code numbers and will call sendSMS to notify us. However, we will have to delay each url request by importing random and time because requesting the url too fast will get us blocked for ddos attempts

In [0]:
from random import *
import time

def checkSeats(id_list, isDelay=False):
  if isDelay:
    time.sleep(randint(5,15))
    
  URL = 'http://suis.sabanciuniv.edu/prod/bwckschd.p_disp_detail_sched?term_in=201801&crn_in='
  course_no = choice(id_list)
  URL += str(course_no)
  r = requests.get(URL)
  tree = lxml.html.fromstring(r.content)
  elms = tree.find_class('dddefault')

  cnt = 0
  data = [] # CAP , ACTUAL , REM

  for el in elms:
    if el.text_content().isdigit():
      cnt += 1
      data.append(int(el.text_content()))
    if cnt > 2:
      break
  if data[-1] > 0:
    print(data)
    print(('EMPTY SEAT FOUND AT ' + str(course_no)))
    sendSMS(('EMPTY SEAT FOUND AT ' + str(course_no)))
    return course_no
  return 0
  
  

Here is where we will start the search loop, after finding the course that is available we will also need the recitation course code in order to register for the course

In [0]:
looping = False
while not looping:
  looping = checkSeats(CLASSES, True)

looping = int(looping)
print(looping)
if MATH == looping:
  for id in MR:
    checkSeats([id])
elif CS == looping :
for id in CR:
  checkSeats([id])
